# Grafon

### Qué va a analizar nuestro grafo?

Analizar "gente que haya tenido experiencias facultativas similares" -> sacamos el factor temporal

### Qué queremos responder?

Qué electivas curso?

### Cómo es el grafo?

- Nodos: usuarios
- Aristas: cursar misma materia y "les fue parecido" -> -1 (en el final), 4-5, 6-7, 8-10
- Peso: porcentaje de similitud de materia+nota

### Dos formas de armar el análisis
- sólo correr sobre materias electivas
- correr sobre todas las materias, pero filtrar el output por electivas

### Ejemplo de conexión:

X y yo cursamos ...

In [ ]:
import pandas as pd

df = pd.read_pickle('fiuba-map-data.pickle')
df.tail(3)

In [ ]:
categories = {
    4: 0,
    5: 0,
    6: 1,
    7: 1,
    8: 2,
    9: 2,
    10: 2
}

df_rel = df[['Padron', 'materia_id', 'materia_nota']].copy()

# Sacamos materias en final y a cursar
df_rel = df_rel[df_rel['materia_nota'] >= 4]
# Sacamos gente que no le pone la nota a su fiubamap
df_rel['promedio'] = df_rel.groupby('Padron')['materia_nota'].transform('mean')
df_rel = df_rel[df_rel['promedio'] > 5]

df_rel['materia_categoria'] = df_rel['materia_nota'].apply(lambda x: categories[x])
df_simil = pd.merge(df_rel, df_rel, on=['materia_id', 'materia_categoria'])
df_simil = df_simil[df_simil['Padron_x'] != df_simil['Padron_y']]
df_simil = df_simil.reset_index()
df_simil

Esto resultaría en un grafo con pocos nodos y 1407892 de aristas, es decir demasiadas aristas. Por eso se busca un enfoque de unificar aristas mediante algún tipo de criterio y de esta forma tener sólo una arista entre padrones.

In [ ]:
import numpy as np

df_simil_agg = df_simil.groupby(['Padron_x', 'Padron_y']).agg(cant=('materia_id', 'count'))
df_simil_agg = df_simil_agg.reset_index()

df_simil_agg['Padron_min'] = df_simil_agg[['Padron_x', 'Padron_y']].min(axis=1)
df_simil_agg['Padron_max'] = df_simil_agg[['Padron_x', 'Padron_y']].max(axis=1)
df_simil_agg = df_simil_agg.drop_duplicates(['Padron_min', 'Padron_max']).reset_index()

df_simil_agg['inv_cant'] = df_simil_agg['cant'].max() - df_simil_agg['cant'] + 1
df_simil_agg

In [ ]:
df_simil_agg.sort_values('cant', ascending=False).head(25)

In [ ]:
import networkx as nx
G = nx.from_pandas_edgelist(df_simil_agg, 
                            source='Padron_x', 
                            target='Padron_y', 
                            edge_attr='inv_cant',
                            create_using=nx.Graph())

print(G)

In [ ]:
G.edges[('-1', '0000')]

In [ ]:
# Veamos el grafo
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
nx.draw_networkx(G, pos=nx.circular_layout(G), width=0.005, node_size=50, with_labels=False)

## Stats generales del grafo

In [ ]:
print(f"""
  El diámetro de la red: {nx.diameter(G)}
  El grado promedio de la red: {sum([n[1] for n in G.degree()]) / len(G):.2f}
  TODO: Los allegados promedio de la red: 
  Puentes globales: {list(nx.bridges(G))}
""")

## Comunidades

In [ ]:
from networkx.algorithms import community
louvain = community.louvain_communities(G, weight='inv_cant', resolution=1.02)

In [ ]:
len(louvain)

In [ ]:
from setup import PADRON, CARRERA, plan_estudios

def padrones_similares(padron, resolution):
    louvain = community.louvain_communities(G, weight='inv_cant', resolution=resolution)
    return list(filter(lambda x: padron in x, louvain))[0]

def materias_padron(padron):
    return df[(df['Padron'] == padron) & (df['materia_nota'] >= 4)]['materia_id'].values

def sugerir_electivas(padron, resolution=1):
    padrones = padrones_similares(padron, resolution)
    df_sugerencias = df_rel[df_rel['Padron'].isin(padrones)].groupby('materia_id').agg(cant=('materia_id', 'count'))
    df_sugerencias = df_sugerencias[~df_sugerencias.index.isin(materias_padron(padron))]
    
    df_materias = pd.read_json(plan_estudios(CARRERA))
    df_sugerencias = pd.merge(df_sugerencias, df_materias, left_on='materia_id', right_on="id")
    df_sugerencias = df_sugerencias[df_sugerencias['categoria'] == 'Materias Electivas']
    df_sugerencias = df_sugerencias[['id', 'materia', 'creditos', 'cant']].sort_values('cant', ascending=False)
    return df_sugerencias.reset_index(drop=True)

sugerir_electivas(PADRON, 1.03)